In [74]:
## This Cell is related to date preprocessing and is completely copied from original kipf's github code!

import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch,torch.nn,torch.sparse,torch.nn.functional,torch.distributions
from sklearn.metrics import roc_auc_score,average_precision_score
from input_data import load_data
from preprocessing import preprocess_graph, sparse_to_tuple, mask_test_edges
np.set_printoptions(threshold=np.inf)




W_gene      =  pd.read_table( "GBM_normalized/gene.txt", delim_whitespace=True).to_numpy()
W_methy     =  pd.read_table( "GBM_normalized/methy.txt", delim_whitespace=True).to_numpy()
W_mirna     =  pd.read_table( "GBM_normalized/mirna.txt", delim_whitespace=True).to_numpy()
W_truelabel =  pd.read_table( "GBM_normalized/truelabel.txt", delim_whitespace=True).to_numpy()
W_argsort =  pd.read_table( "GBM_normalized/argsort.txt", delim_whitespace=True).to_numpy()

gene      =  pd.read_table( "GBM/GLIO_Gene_Expression.txt", delim_whitespace=True).to_numpy().transpose()
methy     =  pd.read_table( "GBM/GLIO_Methy_Expression.txt", delim_whitespace=True).to_numpy().transpose()
mirna     =  pd.read_table( "GBM/GLIO_Mirna_Expression.txt", delim_whitespace=True).to_numpy().transpose()
truelabel =  pd.read_table( "GBM/GLIO_Survival.txt", delim_whitespace=True).to_numpy()

order = W_argsort.reshape(-1)-1

gene = gene[order]
methy = methy[order]
mirna = mirna[order]
truelabel = truelabel[order]

print(gene.shape)
print(methy.shape)
print(mirna.shape)

def get_nomralized_laplacian(a):
    b = np.diag(np.power(1.0 / a.sum(axis=1), 0.5))
    return (b @ a @ b)




# Some preprocessing
adj = W_gene
features = np.c_[mirna,methy]


# adj = W_methy
# features = np.c_[gene,mirna]


adj_norm = get_nomralized_laplacian(adj)
num_features = features.shape[1]

#pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
#norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)

pos_weight = 1
norm = 1







(215, 12042)
(215, 1305)
(215, 534)


In [83]:
#Model definition Cell


adj_label_tensor = torch.Tensor(adj)
adj_norm_tensor  = torch.Tensor(adj_norm)
features_tensor  = torch.Tensor(features)

# from the previous cell following arrays are obtained : 
#adj_label
#adj_norm
#features
#val_edges_false
#test_edges
#test_edges_false




class Model(torch.nn.Module):

    def __init__(self,first_layer_dim=2000,embedding_dim=2000,A_tilda=None ,**kwargs):
        super(Model, self).__init__(**kwargs)
        
        

        self.W0 = torch.nn.Linear(num_features, first_layer_dim)
        self.W1_mean = torch.nn.Linear(first_layer_dim, embedding_dim)
        self.W1_log_std  = torch.nn.Linear(first_layer_dim, embedding_dim)

        self.A_tilda = A_tilda

        self.normal_dist = torch.distributions.MultivariateNormal(loc=torch.zeros(embedding_dim),scale_tril=torch.eye(embedding_dim))

        self.recon = None
        
        
        self.W2 = torch.nn.Linear(embedding_dim,first_layer_dim)
        self.W3 = torch.nn.Linear(first_layer_dim,num_features)
        
        
        self.optimizer = torch.optim.Adam(params=list(self.W0.parameters())+list(self.W1_mean.parameters())+list(self.W1_log_std.parameters())+list(self.W2.parameters())+list(self.W3.parameters()),lr=0.01)


        

    def train(self,x,A):


        first_layer = torch.nn.functional.relu(torch.matmul(self.A_tilda,self.W0(x)))
        z_mean      = torch.matmul(self.A_tilda,self.W1_mean(first_layer))
        z_log_std   = torch.matmul(self.A_tilda,self.W1_log_std(first_layer))


        z = z_mean +self.normal_dist.sample((x.shape[0],)) * torch.exp(z_log_std)
        
        
        ######
        #second_layer  = torch.nn.functional.relu(torch.matmul(self.A_tilda,self.W2(z)))
        second_layer  = torch.nn.functional.relu(self.W2(z))


        x_prim = self.W3(second_layer)
        
        recon_x_xprim_loss = ((x_prim-x)**2).sum(dim=1).mean(dim=0)
        recon_loss= recon_x_xprim_loss/(x_prim.shape[1])*100
        ######

        recon = torch.matmul(z,z.transpose(dim0=0,dim1=1))
        self.recon = torch.sigmoid(recon)

        #recon_loss = (  A    *    torch.nn.functional.logsigmoid(recon)  ).mean()*pos_weight + ( (1-A) * torch.nn.functional.logsigmoid(-recon)).mean()
        #recon_loss = -norm*recon_loss

        kl_loss =   torch.mean(torch.sum(1 + 2 * z_log_std - z_mean**2 -torch.exp(z_log_std)**2, 1))
        kl_loss = -(1)*kl_loss

        loss = recon_loss+kl_loss

        print(recon_loss.item(),kl_loss.item())

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()


    




In [84]:
#Training Cell 

my_model = Model(first_layer_dim=30,embedding_dim=15,A_tilda=adj_norm_tensor)


for epoch_counter in range(2000):
    
    epoch_loss = my_model.train(features_tensor,adj_label_tensor)
    
    #print("Epoch:", '%04d' % (epoch_counter + 1), "train_loss=", "{:.5f}".format(epoch_loss))


print('\n',"#########################################################",'\n')






106.63590240478516 0.41612708568573
102.89878845214844 1.842858910560608
102.06069946289062 0.5754601955413818
101.20060729980469 0.5132054686546326
100.87000274658203 0.47559842467308044
100.45172119140625 0.44269704818725586
100.42671966552734 0.3674851059913635
100.23410034179688 0.2614392042160034
100.21419525146484 0.17815035581588745
100.09208679199219 0.13550598919391632
99.92265319824219 0.11665190756320953
99.7020034790039 0.10146665573120117
99.69094848632812 0.08451452851295471
99.53436279296875 0.07410774379968643
99.56246948242188 0.06570376455783844
99.52139282226562 0.05839686840772629
99.4360122680664 0.0535992830991745
99.54464721679688 0.05135766416788101
99.39601135253906 0.04731586202979088
99.31685638427734 0.04034224525094032
99.36749267578125 0.03419806808233261
99.3240966796875 0.03161727637052536
99.34317779541016 0.03167976066470146
99.29301452636719 0.031764574348926544
99.2501220703125 0.031215818598866463
99.2690200805664 0.030545979738235474
99.34825134277

92.43462371826172 2.285862445831299
92.66443634033203 2.2555172443389893
92.57989501953125 2.3250343799591064
92.06877136230469 2.430626392364502
92.2162857055664 2.470027446746826
91.82366943359375 2.533810615539551
91.89852142333984 2.5107672214508057
92.09579467773438 2.455636501312256
92.19891357421875 2.410503387451172
91.9705581665039 2.322277069091797
92.39977264404297 2.230374813079834
92.57683563232422 2.218067169189453
92.70581817626953 2.3005316257476807
91.88525390625 2.3774173259735107
92.27195739746094 2.4070544242858887
92.64479064941406 2.451056957244873
91.92101287841797 2.562272787094116
91.87672424316406 2.6002840995788574
91.67134857177734 2.542248487472534
92.2831802368164 2.4554097652435303
92.73763275146484 2.431051015853882
91.74951171875 2.49811053276062
91.88026428222656 2.4874653816223145
92.42926025390625 2.3699300289154053
92.02555084228516 2.369609832763672
92.02823638916016 2.373795509338379
92.06124877929688 2.436061382293701
92.04740905761719 2.44594359

90.3359603881836 2.6837055683135986
90.85487365722656 2.704240560531616
90.49980163574219 2.7782909870147705
90.2137451171875 2.850957155227661
90.2795181274414 2.8892765045166016
90.67842864990234 2.8723666667938232
90.44056701660156 2.8836288452148438
90.27226257324219 2.9038074016571045
90.39116668701172 2.905083179473877
90.46463012695312 2.961092710494995
90.34442138671875 2.9219415187835693
90.69818115234375 2.919834613800049
90.59590911865234 2.974868059158325
90.01670837402344 3.043931007385254
90.12657165527344 3.01363205909729
89.91934204101562 2.9327614307403564
90.55101013183594 2.8229751586914062
89.81005859375 2.6959362030029297
90.6724624633789 2.518616199493408
91.25999450683594 2.399686336517334
90.80909729003906 2.502246856689453
90.6011962890625 2.702087163925171
90.4981460571289 2.8430609703063965
91.0262680053711 2.88581919670105
90.0438461303711 3.0577332973480225
90.1537094116211 3.2369916439056396
90.49832916259766 3.092942714691162
90.58662414550781 2.975270032

90.02286529541016 3.127147912979126
89.56333923339844 3.235039472579956
89.09426879882812 3.2995522022247314
89.10237884521484 3.318366765975952
89.55693817138672 3.2528562545776367
89.75142669677734 3.1458232402801514
89.17304229736328 3.1006546020507812
89.59705352783203 3.024904727935791
89.56613159179688 2.9784927368164062
89.34848022460938 2.9395980834960938
89.52919006347656 2.87886905670166
89.29841613769531 2.8879568576812744
89.47126770019531 2.982248306274414
89.58740997314453 2.95064377784729
89.91199493408203 2.858412742614746
89.44701385498047 2.908743143081665
89.89169311523438 2.9442551136016846
89.18473815917969 3.0445525646209717
89.18150329589844 3.101900815963745
89.4588623046875 3.1204018592834473
89.79328918457031 3.1277782917022705
89.57333374023438 3.246556520462036
89.47371673583984 3.1980507373809814
90.1155776977539 3.080264091491699
89.48078918457031 3.0987560749053955
89.71564483642578 3.1208877563476562
89.71932220458984 3.0429956912994385
89.97247314453125

89.14019775390625 3.360668420791626
89.10466766357422 3.36181640625
89.06431579589844 3.3813207149505615
88.66862487792969 3.353929042816162
88.95218658447266 3.250685214996338
88.78910827636719 3.1388673782348633
88.90917205810547 3.041311264038086
89.44610595703125 2.9184916019439697
89.15892028808594 2.845750331878662
89.87299346923828 2.8136463165283203
89.0716781616211 2.9281387329101562
89.92608642578125 3.0699303150177
89.71968841552734 3.096992254257202
89.23457336425781 3.153266429901123
89.49395751953125 3.224398136138916
88.56632232666016 3.350654363632202
89.78215026855469 3.379512071609497
89.45430755615234 3.1796770095825195
89.21918487548828 3.148176670074463
88.75432586669922 3.2166850566864014
88.96459197998047 3.255718946456909
89.41714477539062 3.1489007472991943
88.91683197021484 3.094822406768799
89.22744750976562 3.072427749633789
88.85052490234375 3.0865681171417236
89.50599670410156 3.0688745975494385
89.16008758544922 3.1570191383361816
88.48115539550781 3.3349

88.30769348144531 3.590331554412842
89.31594848632812 3.5218236446380615
88.82671356201172 3.500802516937256
88.84713745117188 3.502277135848999
88.76082611083984 3.3502633571624756
88.94902801513672 3.1686837673187256
88.75553894042969 3.1168034076690674
89.138916015625 3.039933443069458
88.97721862792969 2.9293570518493652
89.70572662353516 2.851762533187866
89.81841278076172 3.054867744445801
89.74800109863281 3.1783995628356934
89.88090515136719 3.2628414630889893
89.13894653320312 3.470630168914795
88.57373809814453 3.7918341159820557
89.2425308227539 3.809833526611328
89.91487884521484 3.527850389480591
89.18074035644531 3.3706209659576416
88.9881591796875 3.346076726913452
89.01143646240234 3.1892013549804688
89.62863159179688 2.9479174613952637
89.37714385986328 2.920029401779175
89.12232971191406 2.9991607666015625
88.84600067138672 3.124140501022339
88.5970458984375 3.1052422523498535
88.98580932617188 3.184903621673584
88.86613464355469 3.3141000270843506
88.49209594726562 3

88.64491271972656 3.359781265258789
88.25662994384766 3.379871129989624
88.69186401367188 3.2933409214019775
88.31499481201172 3.3362884521484375
88.65888977050781 3.536454677581787
88.98543548583984 3.5808794498443604
88.02800750732422 3.5278053283691406
88.79867553710938 3.5086848735809326
88.13175964355469 3.5432021617889404
88.20767974853516 3.487088918685913
88.4751968383789 3.373051166534424
88.62532043457031 3.31286883354187
88.44661712646484 3.316704034805298
88.02099609375 3.3039863109588623
88.13766479492188 3.251720905303955
89.14030456542969 3.196136951446533
88.08148193359375 3.271061897277832
89.00685119628906 3.2945196628570557
87.927978515625 3.3260598182678223
88.78875732421875 3.2819983959198
88.20036315917969 3.289656400680542
88.43275451660156 3.372854471206665
88.49439239501953 3.3668699264526367
88.49105072021484 3.327340602874756
88.0658950805664 3.440138101577759
87.75066375732422 3.5748634338378906
88.05191040039062 3.580566644668579
88.05923461914062 3.5337076

87.89944458007812 3.393500804901123
87.82576751708984 3.3147244453430176
87.82146453857422 3.273864269256592
87.61531829833984 3.3662948608398438
87.99320220947266 3.3840670585632324
87.53821563720703 3.437756299972534
87.60527038574219 3.5512259006500244
87.44876861572266 3.611074447631836
87.72099304199219 3.5672290325164795
87.51709747314453 3.531571626663208
87.89006042480469 3.462092876434326
88.08582305908203 3.373831272125244
87.36766052246094 3.3851284980773926
87.70130157470703 3.4337594509124756
87.3421630859375 3.5643250942230225
87.61620330810547 3.6347076892852783
87.11918640136719 3.670172929763794
87.89859771728516 3.6457552909851074
87.47802734375 3.638455867767334
87.3313217163086 3.5580191612243652
87.70610809326172 3.449150800704956
87.63876342773438 3.3932671546936035
88.30982208251953 3.4472131729125977
87.31077575683594 3.378852605819702
88.31039428710938 3.2844581604003906
87.58918762207031 3.2946555614471436
88.1345443725586 3.276097297668457
88.52266693115234 3

87.11012268066406 3.597653865814209
86.97251892089844 3.6480841636657715
87.53680419921875 3.63568377494812
87.2847671508789 3.6349759101867676
87.106689453125 3.615689754486084
87.39070129394531 3.5642406940460205
87.95113372802734 3.3663647174835205
87.37140655517578 3.3107035160064697
87.48701477050781 3.382845640182495
87.51189422607422 3.51739239692688
87.38671875 3.6026551723480225
87.38089752197266 3.5080208778381348
87.16891479492188 3.5270934104919434
87.27265930175781 3.473715305328369
87.53166961669922 3.437991142272949
87.70030975341797 3.323812484741211
87.85161590576172 3.291332960128784
86.96644592285156 3.2888691425323486
87.65738677978516 3.329040288925171
87.29437255859375 3.2376043796539307
88.08780670166016 3.180088520050049
87.6789779663086 3.297314405441284
87.50377655029297 3.452681541442871
87.47798156738281 3.563114643096924
86.745361328125 3.6752657890319824
87.37907409667969 3.757709503173828
86.98970031738281 3.7840566635131836
87.5340576171875 3.86198067665

In [4]:
print(my_model.recon)

tensor([[0.8781, 0.6112, 0.4715,  ..., 0.6196, 0.5075, 0.3729],
        [0.6112, 0.7133, 0.4720,  ..., 0.6187, 0.5530, 0.5472],
        [0.4715, 0.4720, 0.7687,  ..., 0.5776, 0.4099, 0.3930],
        ...,
        [0.6196, 0.6187, 0.5776,  ..., 0.7642, 0.5344, 0.5138],
        [0.5075, 0.5530, 0.4099,  ..., 0.5344, 0.9167, 0.4891],
        [0.3729, 0.5472, 0.3930,  ..., 0.5138, 0.4891, 0.7880]],
       grad_fn=<SigmoidBackward>)


In [17]:
np.savetxt("gvae_fusion_results/res.txt",my_model.recon.detach().numpy(),delimiter='\t')